# When you have Multiple Access Points

If you have access to multiple access points and their associated signal strength then you can triangulate a position based upon that information with several online APIs. This is one of the ways our phones and web browsers will identify our current location. You may have seen a message from your MAPS app on your phone that says something like "This works better if you turn on your wireless". If you obtain multiple access points that the laptop has been near you can use that same technique to identify its position.

### Unmasking VPNed and spoofed IPs with active collection
First, let's discuss how to obtain artifacts that include multiple access points. One way would be to persuade visitors to a website that will actively query this information and give it to you. This is the technique used by the honeybadger project.  When a visitor to the website opens an office document, java applet, .HTA file or downloads and runs a Powershell script it queries the wireless information in the background and submits it to the website.  It gets the wireless information by running the command below.  Give this a try!   If you have wireless capabilities on your current Windows device run the following in a Windows Command prompt.
```
"netsh wlan show networks mode=bssid | findstr "SSID Signal Channel"
```

You see it shows a list of all the access points that are near your computer and their signal strengths. Honeybadger collects this data and uses it to identify the individual's location.   Because this is based on the physical location of Wireless Access Points and not an IP address that can be spoofed or hidden behind a VPN this is a reliable way to identify the location of a device that is actively trying to hide their location.
The "HoneyBadger" project can be downloaded for free here: https://github.com/lanmaster53/honeybadger
The Setup of the Honeybadger server can be a little overwhelming so Tim Tomes also offers HoneyBadger as a paid subscription service.  https://hb.lanmaster53.com/
This tool does, after asking for permission, execute the command above on the targets machine.  You will want to **check with your legal counsel** before using this tool.
Let me do a quick demo of the paid service.
https://hb.lanmaster53.com/

### Looking up APs found in forensics images
You can also just collect these artifacts as part of a forensics investigation.  We can easily pick up a few access points from the registry, SRUM and other Windows artifacts.  This information also appears in the windows event logs when "Diagnose my network" buttons are clicked.  When you run the diagnostic it generates several events with an ID of 6100. One of those 6100 events has a list of all the wireless access points and signal strengths at the time of the diagnostic.

![alt text](images/Event6100.jpg "Event 6100")


Take that Event 6100 information and triangulate it using Googles API.  This next cell will put the text from a 6100 event into a variable named log_data.   We will pretend that you have used your awesome SEC573 skills to retrieve it from an event log on the target computer.



In [ ]:
log_data = """
List of visible access point(s): 17 item(s) total, 17 item(s) displayed
        BSSID		BSS Type PHY	Signal(dB)	Chnl/freq    SSID
-------------------------------------------------------------------------
1E-8D-CB-84-FA-DF	Infra	 <unknown>	-62		5745000	 ARFF
06-8D-CB-84-FA-DF	Infra	 <unknown>	-63		5745000	 ARAwifi1
0A-8D-CB-84-FA-DF	Infra	 <unknown>	-62		5745000	 AirportPrivateWiFi
0E-8D-CB-84-FA-DF	Infra	 <unknown>	-63		5745000	 Airport_Wifi
02-8D-DB-84-FA-DF	Infra	 <unknown>	-44		11	 (Unnamed Network)
1E-8D-DB-84-FA-DF	Infra	 <unknown>	-44		11	 ARFF
06-8D-DB-84-FA-DF	Infra	 <unknown>	-44		11	 ARAwifi1
0A-8D-DB-84-FA-DF	Infra	 <unknown>	-44		11	 AirportPrivateWiFi
0C-8D-DB-84-FA-DF	Infra	 <unknown>	-42		11	 Airport_Wifi
F4-39-09-61-0E-1F	Infra	 <unknown>	-74		6	 DIRECT-1E-HP OfficeJet Pro 8710
B2-C1-9E-51-BE-D3	Infra	 <unknown>	-51		6	 ATT-WIFI-3191
FA-E4-F0-BE-E3-01	Infra	 <unknown>	-58		6	 Esneepez
54-75-D0-84-2E-F9	Infra	 g	-54		4	 Tailwinds Wi-Fi
54-75-D0-84-2E-F8	Infra	 g	-54		4	 (Unnamed Network)
5C-B0-66-D8-07-F6	Infra	 <unknown>	-57		1	 SBG6580-2-FD39B
6A-E7-C2-C5-D2-BD	Infra	 <unknown>	-60		1	 Samsung Galaxy_8515
0C-8C-24-6F-9E-28	Infra	 <unknown>	-84		10	 SNRS_6F9E28
"""


First we will use some regular expressions which are covered extensively in SEC573 to extract all of the BSSID and signal strengths from the text in the Event Log.  Execute the next cell to extract the data from the log_data variable set in the previous cell.


In [ ]:
import re
ap_data = re.findall(r"([\dA-F-]+)\s+?Infra.*?(-\d+)", log_data, re.MULTILINE)
ap_data


Next we build the JSON data structure that Googles API is expecting us to submit and we make the request. In this data structure we define several "macAddress" and "signalStrength" fields in a JSON record that matches the specification for Google's API.


In [ ]:
ap_list = []
for mac,signal in ap_data:
    ap_list.append({"macAddress":mac ,"signalStrength":signal})
google_data = {"considerIP": "false", "wifiAccessPoints": ap_list}
google_data



This data structure can now be submitted to googles API. But you have to have an API key.  To get an API key you have to provide google with a Credit Card. Rather than asking you to do this I'm going to give you my API key which is associated with my personal credit card. So please be kind and limit your request to those required for todays lab. Once we get back the record we extract the latitude and longitude and generate a google maps link. After executing the next cell you can open the google maps link in another window to see where the laptop was when the event 6100 was generated.


In [ ]:
import requests

#First borrow Mark Baggetts personal API key (please be kind)
#Change the IP to the one provided in class
key = '<copy and paste marks personal API key here>'

#Then generate the request
url = f'https://www.googleapis.com/geolocation/v1/geolocate?key={key}'    
response = requests.post(url=url, json=google_data)    

#Then extract the data and print a google maps link
loc_record = response.json()
lat = loc_record.get("location").get("lat")
lng = loc_record.get("location").get("lng")
maps_url = f"http://maps.google.com/maps?q={lat:0>3.9f},{lng:0>3.9f}&z=15"
print(loc_record.get("accuracy"))
print(maps_url)



That's amazing. It has an accuracy of approximately 20 meters. But what if you are not lucky enough to find an event 6100. They are in fact rather rare to find on a forensics investigation. What if rather than an event 6100 you are only able to find two BSSIDs in the registry? At a minimum, google API will allow you to submit two BSSIDs with no signal strength. Try it to see what happens.


In [ ]:
ap_data = [('0C-8C-24-6F-9E-28'), ('0C-8D-DB-84-FA-DF')]
ap_list = []
for mac in ap_data:
    ap_list.append({"macAddress":mac})
                                   
response = requests.post(url=url, json= {"considerIP": "false", "wifiAccessPoints": ap_list})    
loc_record = response.json()
lat = loc_record.get("location").get("lat")
lng = loc_record.get("location").get("lng")
maps_url = f"http://maps.google.com/maps?q={lat:0>3.9f},{lng:0>3.9f}&z=15"
print(loc_record.get("accuracy"))
print(maps_url)



Our accuracy fell way down.  As a matter a fact, without the Signal strength it appears that google will always give you an accuracy of 150 meters.

Perhaps 150 meters is good enough. If not we can submit a fake signal strength to force googles algorithms to give us a better location. Most of the time the signal strength is somewhere between -40 and -90.  Anecdotally I believe the average is round -60. By setting an arbitrary signal strength of -60 we force google to choose the most accurate point that is equal distance from the access points.

This time I will put this into a function that you can call.  The function find_location() defined below requires that you pass it a list of BSSIDs and the google API key.  It returns back to you both the accuracy and a URL that you can use to see the location.

In the next cell we create the find_location() function and we call it with two sets of wireless APs.  The first set is pulled from the event 6100 above. The second set is made up of access points that shouldn't exist anywhere.  Let's run this and look at the results.




In [ ]:
def find_location(list_of_aps, key):
    url = f'https://www.googleapis.com/geolocation/v1/geolocate?key={key}'       
    ap_list = []
    for mac in list_of_aps:
        ap_list.append({"macAddress":mac ,"signalStrength":"-60"})
    google_data = {"considerIP": "false", "wifiAccessPoints": ap_list}
    response = requests.post(url=url, json=google_data)    
    loc_record = response.json()
    lat = loc_record.get("location").get("lat")
    lng = loc_record.get("location").get("lng")
    accuracy = loc_record.get('accuracy')
    return accuracy, f"http://maps.google.com/maps?q={lat:0>3.9f},{lng:0>3.9f}&z=15"

print("Here is a good response from google.")
ap_data = [('0C-8C-24-6F-9E-28'), ('0C-8D-DB-84-FA-DF')]
the_accuracy,the_url = find_location(ap_data, key)
print(f"The accuracy is {the_accuracy}")
print(the_url)

print("\n\nHere is a response from google when we send it made up BSSIDs.")
ap_data = [('11-22-33-44-55-66'), ('AA-BB-CC-DD-EE-FF')]
the_accuracy,the_url = find_location(ap_data, key)
print(f"The accuracy is {the_accuracy}")
print(the_url)




Our accuracy went back up for the two access points.  It is almost as good as when we submitted a half dozen access points. But what about that second link? Was google able to geolocation our imaginary made up access points?  Where does that one lead? It points at you!

When google doesn't have any information about the Wireless access point pairs you provide it gives you a link to your current location based upon the IP address you are submitting it from. This is the "I don't know" answer from google.  You need to detect that your script.  Don't try to do it based on the accuracy.  The accuracy will change depending upon your location. Instead submit one request with made up BSSIDs and check to see if the location in all future responses match that. Using this technique we can discover access point locations when google knows about any two AP's we manage to find on the system. You just have to pair up the Access Points.

Consider this scenario.  You find artifacts on the laptop that indicate it was in the vicinity of 5 access points.   We will call them AP1, AP2, AP3, AP4 and AP5.  Some of those APs may be in far off distant locations that google cannot triangulate but other may be close enough to one another to establish a location. 

![alt text](images/aps.png "AP Locations")


### We ask google about locations.

We ask: What's the location of AP1 and AP2?

Google: Here is a link to your current location (ie I don't know)

We ask: What's the location of AP1 and AP3?

Google: Here is a link to your current location

We ask: Whats the location of AP1 and AP4?

Google: Google gives a location which by itself would be somewhere along the blue line.

We ask: What's the location of AP1 and AP5?

Google: Here is a link to your current location

We ask: What's the location of AP2 and AP3?

Google: Here is a link to your current location

We ask: What's the locationa of AP2 and AP4?

Google: Google gives you a location which by itself would be somewhere along the green line. 

Now, by looking at the intersection of these sets we can identify that AP4 has a relationship with both AP2 and AP1. Then ...

We ask: What is the location of AP1, AP2, and AP4?

Google: Google gives us a location somewhere close to the purple square in the middle.

Repeating this process we can find locations that would otherwise be inaccessible through most APIs.

### Building our sets of APs
Now, with just a disconnected set of Wireless APs that we find in various artifacts on the computer we can build a list of AP pairs and find their locations. Python also makes building combinations of pretty simple by calling itertools.combinations. Try this next cell to see how you would do that.


In [ ]:
import itertools

list_of_aps = ['AP1','AP2','AP3','AP4','AP5']
list(itertools.combinations(list_of_aps,2))